In [1]:
%qtconsole

In [2]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.gaussian_process.kernels import (RBF, Matern, RationalQuadratic,
ExpSineSquared, DotProduct,ConstantKernel)
from patsy import dmatrix
from time_rescale import TimeRescaling
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from src.parameters import ANIMALS
from loren_frank_data_processing import (make_epochs_dataframe,
                                         make_tetrode_dataframe,
                                         make_neuron_dataframe,
                                         get_spike_indicator_dataframe,
                                         get_interpolated_position_dataframe,
                                         get_LFP_dataframe)

/Users/edeno/miniconda3/envs/modeling_pfc_ca1_neurons/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
epoch_info = make_epochs_dataframe(ANIMALS)
epoch_info

environment   type  exposure
animal day epoch                             
HPa    1   1        presleep  sleep       NaN
           2             lin    run       1.0
           3             nan   rest       NaN
           4            wtr1    run       1.0
           5             nan   rest       NaN
           6            wtr1    run       2.0
           7       postsleep  sleep       NaN
       2   1        presleep  sleep       NaN
           2            wtr1    run       3.0
           3             nan   rest       NaN
           4            wtr1    run       4.0
           5       postsleep  sleep       NaN
       3   1        presleep  sleep       NaN
           2            wtr1    run       5.0
           3             nan   rest       NaN
           4            wtr1    run       6.0
           5       postsleep  sleep       NaN
       4   1        presleep  sleep       NaN
           2            wtr1    run       7.0
           3             nan   rest       NaN
           4            wtr1    run       8.0
           5       postsleep  sleep       NaN
       5   1        presleep  sleep       NaN
           2            wtr1    run       9.0
           3             nan   rest       NaN
           4            wtr1    run      10.0
           5       postsleep  sleep       NaN
       6   1        presleep  sleep       NaN
           2            wtr1    run      11.0
           3             nan   rest       NaN
           4            wtr2    run       1.0
           5       postsleep  sleep       NaN
       7   1        presleep  sleep       NaN
           2            wtr1    run      12.0
           3             nan   rest       NaN
           4            wtr2    run       2.0
           5       postsleep  sleep       NaN
       8   1        presleep  sleep       NaN
           2            wtr1    run      13.0
           3             nan   rest       NaN
           4            wtr2    run       3.0
           5       postsleep  sleep       NaN

In [4]:
epoch_key = ('HPa', 6, 2)

In [5]:
tetrode_info = make_tetrode_dataframe(ANIMALS).xs(epoch_key, drop_level=False)
tetrode_info

area    depth descrip  numcells  \
animal day epoch tetrode_number                                    
HPa    6   2     1                CA1  [[114]]  riptet         7   
                 2                CA1  [[121]]     NaN         0   
                 3                CA1   [[90]]  CA1Ref         0   
                 4                CA1  [[117]]  riptet         8   
                 5                CA1  [[117]]  riptet         5   
                 6                CA1  [[110]]  riptet         0   
                 7                CA1  [[114]]  riptet         0   
                 8               iCA1  [[114]]  riptet         1   
                 9               iCA1  [[101]]  riptet         1   
                 10              iCA1   [[98]]     NaN         1   
                 11              iCA1  [[108]]  riptet         1   
                 12              iCA1  [[116]]  riptet         1   
                 13              iCA1  [[120]]     NaN         0   
                 14              iCA1  [[105]]  riptet         1   
                 15               PFC  [[120]]     NaN         0   
                 16               PFC  [[123]]     NaN         0   
                 17               PFC  [[123]]     NaN         1   
                 18               PFC  [[112]]     NaN         1   
                 19               PFC  [[130]]     NaN         0   
                 20               PFC  [[109]]     NaN         0   

                                    tetrode_id  
animal day epoch tetrode_number                 
HPa    6   2     1               HPa_06_02_001  
                 2               HPa_06_02_002  
                 3               HPa_06_02_003  
                 4               HPa_06_02_004  
                 5               HPa_06_02_005  
                 6               HPa_06_02_006  
                 7               HPa_06_02_007  
                 8               HPa_06_02_008  
                 9               HPa_06_02_009  
                 10              HPa_06_02_010  
                 11              HPa_06_02_011  
                 12              HPa_06_02_012  
                 13              HPa_06_02_013  
                 14              HPa_06_02_014  
                 15              HPa_06_02_015  
                 16              HPa_06_02_016  
                 17              HPa_06_02_017  
                 18              HPa_06_02_018  
                 19              HPa_06_02_019  
                 20              HPa_06_02_020

In [9]:
tetrode_key = ('HPa', 6, 2, 5)

In [6]:
neuron_info = make_neuron_dataframe(ANIMALS).xs(epoch_key, drop_level=False)
neuron_info

area       csi   meanrate  \
animal day epoch tetrode_number neuron_number                              
HPa    6   2     1              1               CA1  0.004049   0.409959   
                                2               CA1  0.117225   0.346888   
                                3               CA1  0.042945   0.405809   
                                4               CA1  0.104167   0.159336   
                                5               CA1  0.091950   2.247303   
                                6               CA1  0.093514   0.550207   
                 4              1               CA1  0.206349   0.156846   
                                2               CA1  0.088235   0.112863   
                                3               CA1  0.023166   0.214938   
                                4               CA1  0.151860   1.092946   
                                5               CA1  0.154930   0.235685   
                                6               CA1  0.047794   0.451452   
                                7               NaN       NaN        NaN   
                                8               NaN       NaN        NaN   
                 5              1               CA1  0.267584   0.542739   
                                2               CA1  0.097618   2.473859   
                                3               NaN       NaN        NaN   
                                4               CA1  0.195122   0.544398   
                                5               CA1  0.069979   2.004149   
                 8              1              iCA1  0.048257   0.309544   
                 9              1              iCA1  0.087662   0.511203   
                 10             1              iCA1  0.067797   0.293776   
                 11             1              iCA1  0.139106   1.169295   
                 12             1              iCA1  0.032258   0.308714   
                 14             1              iCA1  0.087108   0.238174   
                 17             1               PFC  0.205399  56.261411   
                 18             1               PFC  0.006277   1.321992   

                                                       neuron_id  numspikes  \
animal day epoch tetrode_number neuron_number                                 
HPa    6   2     1              1              HPa_06_02_001_001      494.0   
                                2              HPa_06_02_001_002      418.0   
                                3              HPa_06_02_001_003      489.0   
                                4              HPa_06_02_001_004      192.0   
                                5              HPa_06_02_001_005     2708.0   
                                6              HPa_06_02_001_006      663.0   
                 4              1              HPa_06_02_004_001      189.0   
                                2              HPa_06_02_004_002      136.0   
                                3              HPa_06_02_004_003      259.0   
                                4              HPa_06_02_004_004     1317.0   
                                5              HPa_06_02_004_005      284.0   
                                6              HPa_06_02_004_006      544.0   
                                7              HPa_06_02_004_007        NaN   
                                8              HPa_06_02_004_008        NaN   
                 5              1              HPa_06_02_005_001      654.0   
                                2              HPa_06_02_005_002     2981.0   
                                3              HPa_06_02_005_003        NaN   
                                4              HPa_06_02_005_004      656.0   
                                5              HPa_06_02_005_005     2415.0   
                 8              1              HPa_06_02_008_001      373.0   
                 9              1              HPa_06_02_009_001      616.0   
                 10      

In [10]:
neuron_key = ('HPa', 6, 2, 5, 2)

In [13]:
spike = get_spike_indicator_dataframe(neuron_key, ANIMALS)
position_info = get_interpolated_position_dataframe(epoch_key, ANIMALS)
linear_distance = position_info['linear_distance']
x_pos = position_info['x_position']
y_pos = position_info['y_position']
speed = position_info['speed']
head_direction = position_info['head_direction']

In [14]:
eeg = get_LFP_dataframe(tetrode_key, ANIMALS )

In [23]:
eeg.shape

(1807541,)

In [24]:
linear_distance.shape

(1807541,)

In [25]:
x_pos.shape

(1807541,)

In [26]:
y_pos.shape

(1807541,)

In [27]:
speed.shape

(1807541,)

In [28]:
head_direction.shape

(1807541,)

In [29]:
spike.shape

(1807541,)

In [30]:
linear_distance.to_csv('linear_distance.csv', sep=',')
spike.to_csv('spike.csv', sep=',')
x_pos.to_csv('x_position.csv', sep=',')
y_pos.to_csv('y_position.csv', sep=',')
head_direction.to_csv('direction.csv', sep=',')
speed.to_csv('speed.csv', sep=',')
eeg.to_csv('eeg.csv', sep=',')